<a href="https://colab.research.google.com/github/NitCoh/TextAnalysisForBigData/blob/master/Big_Data_Analysis_for_Cyber_Security_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import pandas as pd
import numpy as np

# **Initial Data Exploration**

DATA_COLS:
Columns:
FileNameID,Sha256ID,Md5ID,Sha1ID,IssuerID,SignerID,PublisherID,WebFileIpID,WebFileUrlID,ThreatFamilyID,ThreatNameID,SigSeqID,SignatureNameID,MachineGuidID,RowNumber,PeAttributesString,WebFileUrlDomain,WebFileUrlHostname,Ctph,LsHash,ThreatCategory,ReportTime,SpynetReportGuid,Firewall,Size,Version,Type,SignerHash,IssuerHash,IsValidCertificate,CertInvalidDetails,Checksum,IsRuntimePacked,FileAccess,Screensaver,StartupUserFolder,MarkOfTheWeb,UsnTimestamp,IsInContainer,IsContainer,IsPEFile,IsAutoRunInf,PeTimestamp

Upload the data file!

In [26]:
COLS = "FileNameID,Sha256ID,Md5ID,Sha1ID,IssuerID,SignerID,PublisherID,WebFileIpID,WebFileUrlID,ThreatFamilyID,ThreatNameID,SigSeqID,SignatureNameID,MachineGuidID,RowNumber,PeAttributesString,WebFileUrlDomain,WebFileUrlHostname,Ctph,LsHash,ThreatCategory,ReportTime,SpynetReportGuid,Firewall,Size,Version,Type,SignerHash,IssuerHash,IsValidCertificate,CertInvalidDetails,Checksum,IsRuntimePacked,FileAccess,Screensaver,StartupUserFolder,MarkOfTheWeb,UsnTimestamp,IsInContainer,IsContainer,IsPEFile,IsAutoRunInf,PeTimestamp"
cols_names = COLS.split(",")
cols_names
len(cols_names)

43

In [27]:
## Upload the data file first!
data_path = '/content/Obf_oneInTenWeek1_d1.tsv'

df = pd.read_csv(data_path,sep='\t', names=cols_names)
df.head(n=15)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (41) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,FileNameID,Sha256ID,Md5ID,Sha1ID,IssuerID,SignerID,PublisherID,WebFileIpID,WebFileUrlID,ThreatFamilyID,ThreatNameID,SigSeqID,SignatureNameID,MachineGuidID,RowNumber,PeAttributesString,WebFileUrlDomain,WebFileUrlHostname,Ctph,LsHash,ThreatCategory,ReportTime,SpynetReportGuid,Firewall,Size,Version,Type,SignerHash,IssuerHash,IsValidCertificate,CertInvalidDetails,Checksum,IsRuntimePacked,FileAccess,Screensaver,StartupUserFolder,MarkOfTheWeb,UsnTimestamp,IsInContainer,IsContainer,IsPEFile,IsAutoRunInf,PeTimestamp
0,7689280,17046923,10489946,9382543,9641,46623,188146,462213,5905091,2720,5644,1036378,227,18220780,14467170,NaN,mediafire.com,download2089.mediafire.com,98304:H5y4BLvb5sTOKPg6JqJTkIMMUaVlebVwaccuyPNA...,77e66aabaf5adeaf55b5adb95f5bdfbdb6bf66eefd7d99...,NaN,1/1/2017 7:18:01 PM,528e69bc-405f-4713-a1f6-36c1903116d0,NaN,7041700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,internet,NaN,NaN,NaN,False,NaN,NaN
1,7689292,1024138,18599752,17052884,9641,46623,188146,100848,15003067,2720,5644,24632,616,20983989,16606030,"1,5,21,23,45,84,85,89,94,98,106,109,110,113,11...",mpgh.net,www.mpgh.net,3072:Gnw+4gXWKP0XzBwWZzqZBcsmhVnxC3eClBFydr:6J...,b7bafa5fa7a5d5ee77bee55f6765e55faee6e96b5f95f5...,NaN,1/1/2017 10:22:19 PM,fd4ef558-8f17-4366-9d5f-abc625582d7f,NaN,101888,NaN,NaN,NaN,NaN,NaN,NaN,0001fb8f,0.0,2,NaN,NaN,internet,1/1/2017 11:22:22 PM,True,NaN,True,NaN,3/6/2016 3:43:13 AM
2,14661295,3242391,4079241,7856388,9641,46623,188146,570877,4648460,2720,5644,1036485,632,26901046,21310830,"1,5,21,23,25,31,45,63,73,94,106,109,110,112,11...",pcinspector.de,download.pcinspector.de,12288:FfOSr3qnux6sHMfQFbN7RaWUH56yRebvnIJ94U22...,77e6da7f57bd9dbafdeea65bb7559d6efd56eba76e5555...,NaN,1/1/2017 1:14:55 PM,a371233e-a97c-4a6f-b032-a53403a949ac,NaN,767771,6.10.0.6100,APP,NaN,NaN,NaN,NaN,00000000,1.0,2,NaN,NaN,unknown,1/1/2017 1:14:39 PM,NaN,NaN,True,NaN,9/23/2002 11:55:47 AM
3,7689292,1024138,18599752,17052884,9641,46623,188146,100848,15003067,2720,5644,24632,616,9649625,7582810,"1,5,21,23,45,84,85,89,94,98,106,109,110,113,11...",mpgh.net,www.mpgh.net,3072:Gnw+4gXWKP0XzBwWZzqZBcsmhVnxC3eClBFydr:6J...,b7bafa5fa7a5d5ee77bee55f6765e55faee6e96b5f95f5...,NaN,1/1/2017 11:06:32 PM,65605608-5145-4c86-b76c-573d1b56819a,True,101888,NaN,NaN,NaN,NaN,NaN,NaN,0001fb8f,0.0,2,NaN,NaN,internet,NaN,True,NaN,True,NaN,3/6/2016 3:43:13 AM
4,7689305,1012018,8180486,14860251,9641,46623,188146,955856,2641005,2720,5644,1036641,221,21896518,17330640,NaN,forumprofi.de,clan-z.forumprofi.de,196608:skgNapTLYtSXzUaUykAmSs5iMKrbgdA50eQEvZP...,796a9fb765fbaea5e59e6ff56ea9be67b7a95afd9fe6b5...,NaN,1/1/2017 9:51:00 AM,3c5719d9-8fc9-46c1-b874-c5803cb93b13,True,8254731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,internet,NaN,NaN,NaN,False,NaN,NaN
5,bsplayer_free.exe,6879724,67197,6545231,9618,43582,34,778924,14124668,2720,5644,37,1291,10479025,8249640,"1,5,21,23,25,30,45,46,49,51,60,73,84,86,89,90,...",deliverynowsend.com,www.deliverynowsend.com,24576:5QvY1H02hJ/X99M3suMBWom0cOGNrOhR35qjrmKy...,ebd5b97a77eeeab77aeed66b97d56d7adea77a7d7db9a5...,NaN,1/1/2017 5:48:40 PM,413e9be0-4214-4855-bc79-0ad424ca156d,True,1274648,0.0.0.0,APP,7c3b2769a9c1f667aec9e680f2b144b32edc4f4a,d00cfdbf46c98a838bc10dc4e097ae0152c461bc,True,NaN,9e22bfbd,1.0,2,NaN,NaN,unknown,NaN,NaN,NaN,True,NaN,6/19/1992 10:22:17 PM
6,14661402,17791699,4947414,303038,9520,37557,188146,531297,4584534,2720,5644,1036188,621,28921603,22959790,"1,3,5,21,23,25,30,37,43,45,46,60,61,63,69,94,9...",informer.com,download.informer.com,98304:cYnZTpoC0jyTnWsf8LvjtIU2Tif2dbRY5r6RdS1k...,7bda65bfab6e59a59baad96b7796995e9df66ffa5f6575...,NaN,1/1/2017 9:15:43 PM,a8c687e5-024f-46e9-90ae-e6f67519146e,True,6491376,NaN,NaN,f5bdef563a7620c4fe52d6e2027dd709be3c07c3,495847a93187cfb8c71f840cb7b41497ad95c64f,True,NaN,00635400,1.0,2,NaN,NaN,internet,NaN,NaN,NaN,True,NaN,7/23/2009 8:54:42 AM
7,12566040,11886579,8845633,11421755,183,362,188146,1037662,878,2720,5644,1535,1334,3008239,2353550,"1,3,5,21,23,25,31,43,45,63,77,85,106,109,111,1...",NaN,canonicalizer.uc

In [28]:
df.shape

(205791, 43)

In [0]:
# df2 = df[df.FileNameID]

Textual features:
FileNameID, WebFuleUrlDomain, WebFileUrlHostname.

***FileNameID:***
This column should be used, because it can indicate couple of things about the file itself.
First, the suffix of the file might imply about the maliciousness of the file, for example, 'exe' files are surely more suspicious, because a malicious code might run while executing those files.
Second, the file name itself can raise the suspicious. Common knowledge from surfing the web indicates that attackers might use well-known software names for their malicious files for example "bsplayer_pro270.1080.exe". (entry number 8 at the dataset)

***WebFuleUrlDomain:***
Suspicious domains might lead to rais the probability that the file is malicious, even though attackers sometimes use common well-known domains to upload their files, such as Dropbox, GoogleDrive, MediaFire in order to pass some security checks.

***WebFileUrlHostname***
Sometimes the domain (the network itself) might be non-suspicious, but a machine through this domain (the hostname) can produce a lot of malicious files, for example :
"mediafire.com" and the hostname: "download1507.mediafire.com".
With the cloud storage services today, attackers might hide behind those distributed storage services and the hostname should be considered.

# **Text analysis features:**

First for each text feature chosen, we should each entry in the dataset through a pipeline of normalizations before analysis:
1. Removing digits - some files might contain their versions in their names.
2. Casing - for widely conventions we shall use lower-case.
3. Stop words - replacing each stopword with sepcial char in order to increase the reliability of td-idf method.

In [0]:
#Taken from https://gist.github.com/sebleier/554280 -> NLTK english stop words
stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your",
              "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", 
              "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what",
              "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were",
              "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a",
              "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by",
              "for", "with", "about", "against", "between", "into", "through", "during", "before", "after",
              "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
              "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both",
              "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same",
              "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [0]:
def replace_digits(corpus):
  pass

def lower_case(corpus):
  pass

def replace_stop_words(corpus, stop_words):
  pass

Aggregating the data:
Some files might appear on few entries in the dataset, using the sha1ID,
we can recongize them along the dataset and extract a list of names which this file was known for.

In [0]:
def collect_names(sha1):
  pass

def collect_hostnames(sha1):
  pass

def collect_domains(sha1):
  pass

# **Textual features analysis:**

As we know, the sha1 is hash function working on the content of the file,
so here is the histogram in order to see which files are the most frequent (maybe with different names with the same sha1) in the dataset reports.

In [0]:
df["Sha1ID"].hist()

Analyzing the threat family of the reports, we can see that most of the reports include the same threat name or threat family, because of the anonymization we can tell which one.

In [0]:
df["ThreatNameID"].hist() 

In [0]:
df["ThreatFamilyID"].hist()
